In [5]:
!pip install selenium

  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
   ---------------------------------------- 0.0/9.7 MB ? eta -:--:--
    --------------------------------------- 0.1/9.7 MB 1.7 MB/s eta 0:00:06
   - -------------------------------------- 0.4/9.7 MB 2.8 MB/s eta 0:00:04
   -- ------------------------------------- 0.7/9.7 MB 3.8 MB/s eta 0:00:03
   --- ------------------------------------ 0.9/9.7 MB 4.5 MB/s eta 0:00:02
   ------ --------------------------------- 1.6/9.7 MB 6.4 MB/s eta 0:00:02
   ------------ --------------------------- 3.1/9.7 MB 10.3 MB/s eta 0:00:01
   ---------------------- ----------------- 5.6/9.7 MB 16.2 MB/s eta 0:00:01
   ------------------------- -------------- 6.3/9.7 MB 17.5 MB/s eta 0:00:01
   --------------------------------- ------ 8.2/9.7 MB 18.

ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchaudio 2.2.2 requires torch==2.2.2, but you have torch 2.0.1 which is incompatible.
torchvision 0.17.2 requires torch==2.2.2, but you have torch 2.0.1 which is incompatible.


In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [2]:
url = "https://www.litcharts.com/lit/heart-of-darkness/part-1"
headers = {"User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"}

In [8]:
response = requests.get(url, headers=headers)
print(response.status_code)
soup = BeautifulSoup(response.text, "html.parser")

summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")


200


In [9]:
summary_texts = [summary.text.strip() for summary in summaries]
analysis_texts = [analyse.text.strip() for analyse in analysis]
HoD = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts})
HoD.to_csv("HoD.csv", index=False)


In [10]:
# print first row of the dataframe
print(HoD.head(1))

                                             Summary  \
0  The Narrator describes the scene from the deck...   

                                            Analysis  
0  The opening establishes a dark tone. Water is ...  


In [12]:
part_index = 0

while True:
    part_index += 1
    url = f"https://www.litcharts.com/lit/heart-of-darkness/part-{part_index}"
    response = requests.get(url, headers=headers)
    print(response.status_code)
    if response.status_code != 200:
        break
    soup = BeautifulSoup(response.text, "html.parser")
    summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
    analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
    summary_texts = [summary.text.strip() for summary in summaries]
    analysis_texts = [analyse.text.strip() for analyse in analysis]
    HoD = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts})
    HoD.to_csv("HoD.csv", mode='a', header=False, index=False)

202


In [13]:
print(HoD.head(1))

                                             Summary  \
0  The Narrator describes the scene from the deck...   

                                            Analysis  
0  The opening establishes a dark tone. Water is ...  


In [3]:
import requests
from bs4 import BeautifulSoup

# URL of the page
url = "https://www.litcharts.com/lit/all-my-sons"  # Replace with the actual page URL
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
}

# Send a request to the page
response = requests.get(url, headers=headers)

if response.status_code == 200:
    soup = BeautifulSoup(response.text, 'html.parser')
    
    dropdown_div = soup.find('div', class_='component dropdown summary-sections')
    #print(dropdown_div)

    # Locate the dropdown menu
    dropdown_menu = dropdown_div.find('ul', class_='dropdown-menu')
    
    if dropdown_menu:
        # Extract all <a> tags inside the dropdown menu
        dropdown_items = dropdown_menu.find_all('a')
        
        # Get the text from each <a> tag
        acts = [item.get_text(strip=True) for item in dropdown_items]
        print(acts)  # Output: ['Act 1', 'Act 2', 'Act 3']
    else:
        print("Dropdown menu not found on the page.")
else:
    print(f"Failed to fetch page. Status code: {response.status_code}")


Failed to fetch page. Status code: 202


In [4]:
def get_summary_sections(url):
    response = requests.get(url, headers=headers)
    print(response.status_code)
    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        dropdown_div = soup.find('div', class_='component dropdown summary-sections')
        if dropdown_div:
            dropdown_menu = dropdown_div.find('ul', class_='dropdown-menu')
            if dropdown_menu:
                dropdown_items = dropdown_menu.find_all('a')
                sections = [item.get_text(strip=True).lower().replace(" ", "-") for item in dropdown_items]
                return sections
            else:
                return None
        else:
            return None
    else:
        return None

In [5]:
url = "https://www.litcharts.com/lit/heart-of-darkness"
sections = get_summary_sections(url)
print(sections)

202
None


In [7]:
import time
def get_summary_and_analysis(title, timeout=30):
    url = f"https://www.litcharts.com/lit/{title}"
    sections = get_summary_sections(url)
    time.sleep(timeout)
    if sections:
        for section in sections:
            section_url = f"{url}/{section}"
            response = requests.get(section_url, headers=headers)
            time.sleep(timeout)
            print(response.status_code)
            if response.status_code == 200:
                soup = BeautifulSoup(response.text, "html.parser")
                summaries = soup.find_all("div", class_="summary-text readable highlightable-content non-paywall")
                analysis = soup.find_all("div", class_="analysis-text highlightable-content non-paywall")
                summary_texts = [summary.text.strip() for summary in summaries]
                analysis_texts = [analyse.text.strip() for analyse in analysis]
                testDF = pd.DataFrame({"Summary": summary_texts, "Analysis": analysis_texts})
                testDF.to_csv("test.csv", mode='a', header=False, index=False)

In [6]:
title = "heart-of-darkness"
get_summary_and_analysis(title)

200
200
200
200
